In [3]:
import requests
import os


BASE_URL = "https://openapi.data.uwaterloo.ca/v3"

def register_account(email, project: str = "Flow", uri: str = "https://github.com/AndreyKarmanov/Flow") -> requests.Response:
    url = BASE_URL + "/account/register"
    headers = {
        "content-type": "application/x-www-form-urlencoded"
    }

    data = {
        "email": email,
        "project": project,
        "uri": uri
    }

    # and then just use the curl command in CMD (not powershell) to get activated. The email has the api token. 
    return requests.post(url, headers=headers, data=data)

def get_academic_organizations(api_key):
    url = 'https://openapi.data.uwaterloo.ca/v3/AcademicOrganizations'
    headers = {
        'accept': 'application/json',
        'x-api-key': api_key
    }
    
    response = requests.get(url, headers=headers)
    return response.json()


In [1]:
api_token = os.getenv("UW_TOKEN")

In [4]:
academic_orgs = get_academic_organizations(api_token)

In [5]:
academic_orgs.

[{'code': 'ACC',
  'name': 'ACC',
  'description': None,
  'descriptionFormal': 'Accounting & Finance - School of',
  'associatedCampusCode': 'UW'},
 {'code': 'AMATH',
  'name': 'AMATH',
  'description': None,
  'descriptionFormal': 'Applied Mathematics',
  'associatedCampusCode': 'UW'},
 {'code': 'ANTH',
  'name': 'ANTH',
  'description': None,
  'descriptionFormal': 'Anthropology',
  'associatedCampusCode': 'UW'},
 {'code': 'ARCH',
  'name': 'ARCH',
  'description': None,
  'descriptionFormal': 'Architecture - School of',
  'associatedCampusCode': 'UW'},
 {'code': 'ART',
  'name': 'ART',
  'description': None,
  'descriptionFormal': 'Arts',
  'associatedCampusCode': 'UW'},
 {'code': 'ARTSDEAN',
  'name': 'ARTSDEAN',
  'description': None,
  'descriptionFormal': 'Dean of Arts',
  'associatedCampusCode': 'UW'},
 {'code': 'BIOL',
  'name': 'BIOL',
  'description': None,
  'descriptionFormal': 'Biology',
  'associatedCampusCode': 'UW'},
 {'code': 'CBET',
  'name': 'CBET',
  'description'